1) read the parsed data

In [ ]:
import feedparser
from newspaper import Article


rss_url = 'http://rss.cnn.com/rss/cnn_topstories.rss'
feed = feedparser.parse(rss_url)


for entry in feed.entries:
    article_url = entry.link
    print(f"Fetching article: {entry.title}")

    try:
        
        article = Article(article_url)
        article.download()
        article.parse()
        
       
        print(article.text)
    except Exception as e:
        print(f"Failed to parse the article: {e}")
    
    print()


2) creating necessary tables

In [154]:
import mysql.connector
from mysql.connector import Error

try:
    
    cnx = mysql.connector.connect(
        user='root',
        password='#Likithxlr8',
        host='localhost',  
        database='new_articles',
        auth_plugin='mysql_native_password'
    )

    
    cursor = cnx.cursor()

        
    create_table_query = """
        CREATE TABLE articles_preprocessing (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255) NOT NULL UNIQUE,
    url VARCHAR(255) NOT NULL UNIQUE,
    content TEXT,
    published_date DATETIME
    
    );


        """

       
    cursor.execute(create_table_query)
    print("Table 'articles_preprocessing' created successfully or already exists.")

        
    cursor.close()
        

except Error as e:
    
    print(f"Error: {e}")

try:
   
    cnx = mysql.connector.connect(
        user='root',
        password='#Likithxlr8',
        host='localhost',  
        database='new_articles',
        auth_plugin='mysql_native_password'
    )

    
    cursor = cnx.cursor()

       
    create_table_query = """
        CREATE TABLE articles (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255) NOT NULL UNIQUE,
    url VARCHAR(255) NOT NULL UNIQUE,
    content TEXT,
    published_date DATETIME,
    category VARCHAR(100)
    );


        """

        
    cursor.execute(create_table_query)
    print("Table 'articles' created successfully or already exists.")

       
    cursor.close()
        

except Error as e:
    
    print(f"Error: {e}")







Error: 1050 (42S01): Table 'articles_preprocessing' already exists
Error: 1050 (42S01): Table 'articles' already exists


3) insertion operation

In [149]:
import feedparser
from newspaper import Article
import mysql.connector
from mysql.connector import Error, IntegrityError
from datetime import datetime, timezone
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


db_config = {
    'host': 'localhost',
    'database': 'new_articles',
    'user': 'root',
    'password': '#Likithxlr8',
    'auth_plugin':'mysql_native_password'
}

def insert_article_preprocessing(title, content, url, published_date):
    connection = None  
    try:
        connection = mysql.connector.connect(**db_config)
        if connection.is_connected():
            cursor = connection.cursor()
            insert_query = """
            INSERT INTO articles_preprocessing (title, content, url, published_date)
            VALUES (%s, %s, %s, %s)
            """
            data_tuple = (title, content, url, published_date)
            cursor.execute(insert_query, data_tuple)
            connection.commit()
            print("Article inserted successfully.")
    except IntegrityError as ie:
        print(f"Error: Duplicate entry for title '{title}'. {ie}")
    except Error as e:
        print(f"Error: {e}")
    finally:
        if connection and connection.is_connected():  
            cursor.close()
            connection.close()


def insert_article(title, content, url, published_date,category):
    connection = None  
    try:
        connection = mysql.connector.connect(**db_config)
        if connection.is_connected():
            cursor = connection.cursor()
            insert_query = """
            INSERT INTO articles (title, content, url, published_date,category)
            VALUES (%s, %s, %s, %s, %s)
            """
            data_tuple = (title, content, url, published_date,category)
            cursor.execute(insert_query, data_tuple)
            connection.commit()
            print("Article inserted successfully.")
    except IntegrityError as ie:
        print(f"Error: Duplicate entry for title '{title}'. {ie}")
    except Error as e:
        print(f"Error: {e}")
    finally:
        if connection and connection.is_connected():  
            cursor.close()
            connection.close()



acquiring content

In [ ]:

links_url = ["http://rss.cnn.com/rss/cnn_topstories.rss", "http://qz.com/feed", "http://feeds.foxnews.com/foxnews/politics", "http://feeds.reuters.com/reuters/businessNews", "http://feeds.feedburner.com/NewshourWorld", "https://feeds.bbci.co.uk/news/world/asia/india/rss.xml"]
for links in links_url:
    feed = feedparser.parse(links)
    for entry in feed.entries:
        title = entry.title
        url = entry.link

        
        if hasattr(entry, 'published'):
            published_date = entry.published
        else:
            
            published_date = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

        
        try:
            published_date_obj = datetime.strptime(published_date[:-4], '%a, %d %b %Y %H:%M:%S')
            published_date_obj = published_date_obj.replace(tzinfo=timezone.utc)  # Set timezone to UTC
            published_date_str = published_date_obj.strftime('%Y-%m-%d %H:%M:%S')
        except ValueError as e:
            print(f"Error parsing date: {e}")
            continue  

        
        try:
            article = Article(url)
            article.download()
            article.parse()

            
            content = article.text
            

            
            if content:
                insert_article_preprocessing(title, content, url, published_date_str)
            else:
                print(f"No content found for article: {title}")

        except Exception as e:
            print(f"Error processing article '{title}': {e}")


read from data base (mysql)

In [ ]:
import mysql.connector
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity



def read_data_from_table():
    try:
        
        connection = mysql.connector.connect(
            host="localhost",      
            user="root",
            password="#Likithxlr8",
            database="new_articles",
            auth_plugin='mysql_native_password'
        )

       
        cursor = connection.cursor()

       
        query = "SELECT * FROM articles_preprocessing"

        
        cursor.execute(query)

        
        results = cursor.fetchall()

        
        nlp = spacy.load("en_core_web_md")

        
        categories = {
                "Terrorism/Protest/Unrest": ["terrorism", "protest", "riot", "demonstration", "violence", "unrest", "extremism", "radicalization", "revolution", "insurgency"],
                "Politics": ["politics", "government", "election", "democracy", "parliament", "politician", "policy", "legislation", "campaign", "diplomacy"],
                "Natural Disasters": ["earthquake", "flood", "hurricane", "tornado", "tsunami", "wildfire", "landslide", "volcano", "cyclone", "drought"],
                "Technology": ["innovation", "artificial intelligence", "software", "hardware", "startup", "cloud computing", "cybersecurity", "gadgets", "data", "machine learning"],
                "Business": ["business", "commerce", "enterprise", "startup", "economy", "finance", "investment", "market", "trade", "corporation"],
                "Entertainment": ["celebrity", "movie", "film", "music", "television", "concert", "award", "gossip", "theater", "festival"],
                "Sports":["Soccer (Football)", "Cricket", "Basketball", "Field Hockey", "Tennis", "Volleyball", "Table Tennis", "Baseball", "American Football", "Golf"]

            }

        
        for row in results:
            title = row[1]           
            url = row[2]              
            content = row[3]          
            published_date = row[4]   
            
            
            text = content
            doc = nlp(text)
            doc_vector = doc.vector
            

            def compute_similarity(text_vector, category_keywords, nlp):
                keyword_vectors = [nlp(keyword).vector for keyword in category_keywords]
                mean_keyword_vector = np.mean(keyword_vectors, axis=0)  
                similarity_score = cosine_similarity([text_vector], [mean_keyword_vector])
                return similarity_score[0][0]

           
            category_scores = {}
            for category, keywords in categories.items():
                category_scores[category] = compute_similarity(doc_vector, keywords, nlp)

            max_key = max(category_scores, key=category_scores.get)
            print("Category Scores:", category_scores)
            if content:
                insert_article(title, content, url, published_date,max_key)
            else:
                print(f"No content found for article: {title}")


    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally:
       
        if cursor:
            cursor.close()
        if connection:
            connection.close()


read_data_from_table()
